https://www.appsloveworld.com/opencv/100/29/how-to-do-remove-jagged-edges-of-an-object-in-an-image-with-python-and-opencv

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.interpolate import splprep, splev

# Load the image
im = cv2.imread('image1.jpeg')
bk = im.copy()

# Fill background with black color
cv2.floodFill(im, None, seedPoint=(1, 1), newVal=(0, 0, 0), loDiff=(5, 5, 5), upDiff=(5, 5, 5))
gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)

ret, thresh_gray = cv2.threshold(gray, 5, 255, cv2.THRESH_BINARY)

# Use "open" morphological operation for removing small contours (noise)
thresh_gray = cv2.morphologyEx(thresh_gray, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))

# Background
bk[(thresh_gray > 0)] = 0
bk = cv2.morphologyEx(bk, cv2.MORPH_DILATE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20)))

# Foreground
fg = im.copy()
tmm_fg = cv2.morphologyEx(fg, cv2.MORPH_DILATE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20)))
fg_gray = cv2.cvtColor(fg, cv2.COLOR_RGB2GRAY)
fg[(fg_gray == 0)] = tmm_fg[(fg_gray == 0)]

# Find contours (there is only one contour)
contours, _ = cv2.findContours(thresh_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[-2:]

# Smooth contour
x, y = contours[0].T
x = x.tolist()[0]
y = y.tolist()[0]
tck, u = splprep([x, y], u=None, s=1.0, per=1)
u_new = np.linspace(u.min(), u.max(), 150)
x_new, y_new = splev(u_new, tck, der=0)
res_array = [[[int(i[0]), int(i[1])]] for i in zip(x_new, y_new)]
smoothened = np.asarray(res_array, dtype=np.int32)

# Build a mask
mask = np.zeros_like(thresh_gray)
cv2.drawContours(mask, [smoothened], -1, 255, -1)

# For testing
test_im = cv2.cvtColor(thresh_gray, cv2.COLOR_GRAY2RGB)
cv2.drawContours(test_im, [smoothened], 0, (0, 255, 0), 1)

# Overlay smoothed contour on the original image
res = bk.copy()
res[(mask > 0)] = fg[(mask > 0)]

# Display the images side by side using matplotlib
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

axes[0].imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
axes[0].set_title("Original Image")

axes[1].imshow(cv2.cvtColor(test_im, cv2.COLOR_BGR2RGB))
axes[1].set_title("Smoothed Contour")

axes[2].imshow(cv2.cvtColor(bk, cv2.COLOR_BGR2RGB))
axes[2].set_title("Background Mask")

axes[3].imshow(cv2.cvtColor(res, cv2.COLOR_BGR2RGB))
axes[3].set_title("Result")

plt.show()


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from scipy.interpolate import splprep, splev

def smooth_contour_and_overlay(original_image):
    bk = original_image.copy()

    # Fill background with black color
    cv2.floodFill(original_image, None, seedPoint=(1, 1), newVal=(0, 0, 0), loDiff=(5, 5, 5), upDiff=(5, 5, 5))
    gray = cv2.cvtColor(original_image, cv2.COLOR_RGB2GRAY)

    ret, thresh_gray = cv2.threshold(gray, 5, 255, cv2.THRESH_BINARY)

    # Use "open" morphological operation for removing small contours (noise)
    thresh_gray = cv2.morphologyEx(thresh_gray, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))

    # Background
    bk[(thresh_gray > 0)] = 0
    bk = cv2.morphologyEx(bk, cv2.MORPH_DILATE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20)))

    # Foreground
    fg = original_image.copy()
    tmm_fg = cv2.morphologyEx(fg, cv2.MORPH_DILATE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20)))
    fg_gray = cv2.cvtColor(fg, cv2.COLOR_RGB2GRAY)
    fg[(fg_gray == 0)] = tmm_fg[(fg_gray == 0)]

    # Find contours (there is only one contour)
    contours, _ = cv2.findContours(thresh_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[-2:]

    # Smooth contour
    x, y = contours[0].T
    x = x.tolist()[0]
    y = y.tolist()[0]
    tck, u = splprep([x, y], u=None, s=1.0, per=1)
    u_new = np.linspace(u.min(), u.max(), 150)
    x_new, y_new = splev(u_new, tck, der=0)
    res_array = [[[int(i[0]), int(i[1])]] for i in zip(x_new, y_new)]
    smoothened = np.asarray(res_array, dtype=np.int32)

    # Build a mask
    mask = np.zeros_like(thresh_gray)
    cv2.drawContours(mask, [smoothened], -1, 255, -1)

    # Overlay smoothed contour on the original image
    result_image = bk.copy()
    result_image[(mask > 0)] = fg[(mask > 0)]

    return result_image

# Define the path to the folder containing the images
folder_path = "path/to/images_folder"

# Get a list of all image file names in the folder
image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Process each image and display the results using matplotlib
for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    original_image = cv2.imread(image_path)
    result_image = smooth_contour_and_overlay(original_image)

    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    axes[0].imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
    axes[0].set_title("Original Image")

    axes[1].imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
    axes[1].set_title("Result")

    plt.show()


Color Version

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.interpolate import splprep, splev

# Load the image
im = cv2.imread('image1.jpeg')

# Split the image into individual color channels
b, g, r = cv2.split(im)

# Fill background with black color
cv2.floodFill(b, None, seedPoint=(1, 1), newVal=0, loDiff=5, upDiff=5)
cv2.floodFill(g, None, seedPoint=(1, 1), newVal=0, loDiff=5, upDiff=5)
cv2.floodFill(r, None, seedPoint=(1, 1), newVal=0, loDiff=5, upDiff=5)

# Create grayscale image
gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

ret, thresh_gray = cv2.threshold(gray, 5, 255, cv2.THRESH_BINARY)

# Use "open" morphological operation for removing small contours (noise)
thresh_gray = cv2.morphologyEx(thresh_gray, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))

# Background
b[(thresh_gray > 0)] = 0
g[(thresh_gray > 0)] = 0
r[(thresh_gray > 0)] = 0

bk = cv2.merge((b, g, r))
bk = cv2.morphologyEx(bk, cv2.MORPH_DILATE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20)))

# Foreground
fg = im.copy()
tmm_fg = cv2.morphologyEx(fg, cv2.MORPH_DILATE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20)))
fg_gray = cv2.cvtColor(fg, cv2.COLOR_BGR2GRAY)
fg[(fg_gray == 0)] = tmm_fg[(fg_gray == 0)]

# Find contours (there is only one contour)
contours, _ = cv2.findContours(thresh_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[-2:]

# Smooth contour
x, y = contours[0].T
x = x.tolist()[0]
y = y.tolist()[0]
tck, u = splprep([x, y], u=None, s=1.0, per=1)
u_new = np.linspace(u.min(), u.max(), 150)
x_new, y_new = splev(u_new, tck, der=0)
res_array = [[[int(i[0]), int(i[1])]] for i in zip(x_new, y_new)]
smoothened = np.asarray(res_array, dtype=np.int32)

# Build a mask
mask = np.zeros_like(thresh_gray)
cv2.drawContours(mask, [smoothened], -1, 255, -1)

# Overlay smoothed contour on the original image
res = bk.copy()
res[(mask > 0)] = fg[(mask > 0)]

# Display the images side by side using matplotlib
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

axes[0].imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
axes[0].set_title("Original Image")

axes[1].imshow(cv2.cvtColor(thresh_gray, cv2.COLOR_GRAY2RGB))
axes[1].set_title("Smoothed Contour")

axes[2].imshow(cv2.cvtColor(bk, cv2.COLOR_BGR2RGB))
axes[2].set_title("Background Mask")

axes[3].imshow(cv2.cvtColor(res, cv2.COLOR_BGR2RGB))
axes[3].set_title("Result")

plt.show()


In [2]:
# With all function incluede


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from scipy.interpolate import splprep, splev

# Total Variation Regularization (TVR) function with dilated edges
def total_variation_regularization(image, weight=0.1, n_iter=50):
    edges = cv2.Canny(image, 100, 200)
    dilated_edges = cv2.dilate(edges, None, iterations=2)
    smoothed_image = cv2.ximgproc.dctDenoising(image, None, n_iter, None, weight)
    smoothed_image[dilated_edges > 0] = image[dilated_edges > 0]
    return smoothed_image

# Function to apply an edge smoothing technique and save the result to a folder
def apply_edge_smoothing(original_folder, output_folder, smoothing_function, **params):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(original_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(original_folder, filename)
            original_image = cv2.imread(image_path)

            # Apply the edge smoothing technique
            smoothed_image = smoothing_function(original_image, **params)

            # Save the smoothed image to the output folder
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, smoothed_image)

# Load the image
def load_image(image_path):
    return cv2.imread(image_path)

# Split the image into individual color channels
def split_channels(image):
    return cv2.split(image)

# Fill background with black color
def fill_background_with_black(image, seed_point=(1, 1), lo_diff=(5, 5, 5), up_diff=(5, 5, 5)):
    filled_image = image.copy()
    cv2.floodFill(filled_image, None, seedPoint=seed_point, newVal=0, loDiff=lo_diff, upDiff=up_diff)
    return filled_image

# Create grayscale image
def create_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply "open" morphological operation to remove small contours (noise)
def morphological_open(image, kernel_size=(5, 5)):
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size))

# Apply "dilate" morphological operation
def morphological_dilate(image, kernel_size=(20, 20)):
    return cv2.morphologyEx(image, cv2.MORPH_DILATE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size))

# Find contours in the image
def find_contours(image):
    return cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[-2:]

# Smooth contour
def smooth_contour(contour, num_points=150):
    x, y = contour.T
    x = x.tolist()[0]
    y = y.tolist()[0]
    tck, u = splprep([x, y], u=None, s=1.0, per=1)
    u_new = np.linspace(u.min(), u.max(), num_points)
    x_new, y_new = splev(u_new, tck, der=0)
    res_array = [[[int(i[0]), int(i[1])]] for i in zip(x_new, y_new)]
    smoothened = np.asarray(res_array, dtype=np.int32)
    return smoothened

# Build a mask from the contour
def build_mask(image, contour):
    mask = np.zeros_like(image)
    cv2.drawContours(mask, [contour], -1, 255, -1)
    return mask

# Overlay smoothed contour on the original image
def overlay_contour(image, mask, foreground):
    res = image.copy()
    res[(mask > 0)] = foreground[(mask > 0)]
    return res

# Function to process all images in a folder
def process_images_in_folder(input_folder, output_folder, smoothing_function, **params):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(input_folder, filename)
            original_image = load_image(image_path)

            # Apply the edge smoothing technique
            smoothed_image = smoothing_function(original_image, **params)

            # Save the smoothed image to the output folder
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, smoothed_image)

# Define the input folder containing the original images
input_images_folder = "path/to/original_images_folder"

# Define the output folder for saving the smoothed images
output_images_folder = "path/to/output/smoothed_images"

# Process all images in the input folder and save the smoothed images to the output folder
process_images_in_folder(input_images_folder, output_images_folder, total_variation_regularization, weight=0.1, n_iter=50)


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.interpolate import splprep, splev

# Load the image
im = cv2.imread('image1.jpeg')

# Split the image into individual color channels
b, g, r = cv2.split(im)

# Fill background with black color
cv2.floodFill(b, None, seedPoint=(1, 1), newVal=0, loDiff=5, upDiff=5)
cv2.floodFill(g, None, seedPoint=(1, 1), newVal=0, loDiff=5, upDiff=5)
cv2.floodFill(r, None, seedPoint=(1, 1), newVal=0, loDiff=5, upDiff=5)

# Create grayscale image
gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

ret, thresh_gray = cv2.threshold(gray, 5, 255, cv2.THRESH_BINARY)

# Use "open" morphological operation for removing small contours (noise)
thresh_gray = cv2.morphologyEx(thresh_gray, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))

# Background (white)
b[(thresh_gray > 0)] = 255
g[(thresh_gray > 0)] = 255
r[(thresh_gray > 0)] = 255

bk = cv2.merge((b, g, r))
bk = cv2.morphologyEx(bk, cv2.MORPH_DILATE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20)))

# Foreground
fg = im.copy()
tmm_fg = cv2.morphologyEx(fg, cv2.MORPH_DILATE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20)))
fg_gray = cv2.cvtColor(fg, cv2.COLOR_BGR2GRAY)
fg[(fg_gray == 0)] = tmm_fg[(fg_gray == 0)]

# Find contours (there is only one contour)
contours, _ = cv2.findContours(thresh_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[-2:]

# Smooth contour
x, y = contours[0].T
x = x.tolist()[0]
y = y.tolist()[0]
tck, u = splprep([x, y], u=None, s=1.0, per=1)
u_new = np.linspace(u.min(), u.max(), 150)
x_new, y_new = splev(u_new, tck, der=0)
res_array = [[[int(i[0]), int(i[1])]] for i in zip(x_new, y_new)]
smoothened = np.asarray(res_array, dtype=np.int32)

# Build a mask
mask = np.zeros_like(thresh_gray)
cv2.drawContours(mask, [smoothened], -1, 255, -1)

# Overlay smoothed contour on the original image
res = bk.copy()
res[(mask > 0)] = fg[(mask > 0)]

# Display the images side by side using matplotlib
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

axes[0].imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
axes[0].set_title("Original Image")

axes[1].imshow(cv2.cvtColor(thresh_gray, cv2.COLOR_GRAY2RGB))
axes[1].set_title("Smoothed Contour")

axes[2].imshow(cv2.cvtColor(bk, cv2.COLOR_BGR2RGB))
axes[2].set_title("Background Mask")

axes[3].imshow(cv2.cvtColor(res, cv2.COLOR_BGR2RGB))
axes[3].set_title("Result")

plt.show()


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import splprep, splev

def smooth_contour(image_path):
    # Load the image
    im = cv2.imread(image_path)

    # Split the image into individual color channels
    b, g, r = cv2.split(im)

    # Fill background with black color
    cv2.floodFill(b, None, seedPoint=(1, 1), newVal=0, loDiff=5, upDiff=5)
    cv2.floodFill(g, None, seedPoint=(1, 1), newVal=0, loDiff=5, upDiff=5)
    cv2.floodFill(r, None, seedPoint=(1, 1), newVal=0, loDiff=5, upDiff=5)

    # Create grayscale image
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    ret, thresh_gray = cv2.threshold(gray, 5, 255, cv2.THRESH_BINARY)

    # Use "open" morphological operation for removing small contours (noise)
    thresh_gray = cv2.morphologyEx(thresh_gray, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))

    # Background (white)
    b[(thresh_gray > 0)] = 255
    g[(thresh_gray > 0)] = 255
    r[(thresh_gray > 0)] = 255

    bk = cv2.merge((b, g, r))
    bk = cv2.morphologyEx(bk, cv2.MORPH_DILATE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20)))

    # Foreground
    fg = im.copy()
    tmm_fg = cv2.morphologyEx(fg, cv2.MORPH_DILATE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20)))
    fg_gray = cv2.cvtColor(fg, cv2.COLOR_BGR2GRAY)
    fg[(fg_gray == 0)] = tmm_fg[(fg_gray == 0)]

    # Find contours (there is only one contour)
    contours, _ = cv2.findContours(thresh_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[-2:]

    # Smooth contour
    x, y = contours[0].T
    x = x.tolist()[0]
    y = y.tolist()[0]
    tck, u = splprep([x, y], u=None, s=1.0, per=1)
    u_new = np.linspace(u.min(), u.max(), 150)
    x_new, y_new = splev(u_new, tck, der=0)
    res_array = [[[int(i[0]), int(i[1])]] for i in zip(x_new, y_new)]
    smoothened = np.asarray(res_array, dtype=np.int32)

    # Build a mask
    mask = np.zeros_like(thresh_gray)
    cv2.drawContours(mask, [smoothened], -1, 255, -1)

    # Overlay smoothed contour on the original image
    res = bk.copy()
    res[(mask > 0)] = fg[(mask > 0)]

    return res


def apply_edge_smoothing(original_folder, output_folder, smoothing_function, **params):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(original_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(original_folder, filename)
            result_image = smoothing_function(image_path, **params)

            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, result_image)


# Define the paths for the original images folder and output folder
original_images_folder = "path/to/original_images_folder"
output_folder = "path/to/output_folder"

# Apply the edge smoothing function to all images in the folder
apply_edge_smoothing(original_images_folder, output_folder, smooth_contour)
